In [1]:
import pandas as pd
import numpy as np

In [2]:
#read the large csv file with specified chunksize 
df_chunk = pd.read_csv("//ant/dept-eu/LHR16/Dept4/AIV-TVOD/BI/PMD_Analysis/PMD_data_PMD_cohortRed4.txt", sep='\t', 
                       low_memory = False, encoding = 'ISO-8859-1', chunksize = 1000000, error_bad_lines = False)


In [3]:
def make_prep(df):

    df['transaction_date_local'] = pd.to_datetime(df['transaction_date_local'])
    df['transaction_datetime_local'] = pd.to_datetime(df['transaction_datetime_local'])
    df['first_purchase_date'] = pd.to_datetime(df['first_purchase_date'])
    df['tenure'] = [int(i.days) for i in (max(df['transaction_date_local']) - df['first_purchase_date'])]
    df['cumulative_sum_total'] = df['cumulative_sum_total'].astype(float)
    df['tenure_days_up_to_last_purchase'] = [int(i.days) for i in (df['transaction_date_local']) - df['first_purchase_date']] 
    df['tenure_days_up_to_last_purchase_corrected'] = np.where(df['tenure_days_up_to_last_purchase'] < 0, 0, df['tenure_days_up_to_last_purchase'] )
    
    df['tenure_months_up_to_last_purchase'] = df['tenure_days_up_to_last_purchase'] / 30
    df['tenure_up_to_last_purchase_corrected'] = np.where(df['tenure_months_up_to_last_purchase'] < 1, 1, df['tenure_months_up_to_last_purchase']).astype(float)
    df['arpu_monthly_dynamic'] = df['cumulative_sum_total'] / (df['tenure_up_to_last_purchase_corrected'])
    
    df['frequency'] = df.groupby('encrypted_customer_id')['tenure_days_up_to_last_purchase_corrected'].diff()
    
    pd.set_option('mode.chained_assignment', None)
      # set default value
    df['tenure_years'] = "default"
    df['tenure_years'][(df['tenure'] < 360)] = "under_year"
    df['tenure_years'][(df['tenure'] >= 360) & (df['tenure'] < 360*2)] = 'btw_1and2yrs'
    df['tenure_years'][(df['tenure'] >= 360*2) & (df['tenure'] < 360*3)] = 'btw_2and3yrs'
    df['tenure_years'][(df['tenure'] >= 360*3) & (df['tenure'] < 360*4)] = 'btw_3and4yrs'
    df['tenure_years'][(df['tenure'] >= 360*4) & (df['tenure'] < 360*5)] = 'btw_4and5yrs'
    df['tenure_years'][(df['tenure'] >= 360*5) ] = 'over_5years'
    
    
    df = df.fillna(0)
    
    return df

In [4]:
chunk_list = []  # append each chunk df here 

# Each chunk is in df format
for chunk in df_chunk:  
    # perform data filtering 
    chunk_filter = make_prep(chunk)
    
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk_filter)
    
# concat the list into dataframe 
df_concat = pd.concat(chunk_list)

In [5]:
#df_concat.to_pickle("data_granularDE.pkl")
df_concat.to_pickle("data_granularDE_pmdReduced.pkl")

In [ ]:
#df_concatenated = pd.read_pickle("data_granularDE.pkl")

In [6]:
#df_concatenated.head()
df_concat.head()

,marketplace_id,encrypted_customer_id,unencrypted_customer_id,transaction_date_local,transaction_datetime_local,week_ending,revenue,units,content_age,first_purchase_date,...,running_tenure_ever,running_arpu_monthly_ever,tenure,tenure_days_up_to_last_purchase,tenure_days_up_to_last_purchase_corrected,tenure_months_up_to_last_purchase,tenure_up_to_last_purchase_corrected,arpu_monthly_dynamic,frequency,tenure_years
0,4,A0270938TP78M39R7CN6,9223372034564167251,2017-09-23,2017-09-23 19:38:40,2019-12-14 00:00:00,4.19,1,A_Under_30d,2017-09-23 19:38:40,...,0.0,4.1900,820,-1,0,-0.033333,1.000000,4.190000,0.0,btw_2and3yrs
1,4,A0270938TP78M39R7CN6,9223372034564167251,2017-11-24,2017-11-24 22:59:57,2019-12-14 00:00:00,0.83,1,B_30days_6m,2017-09-23 19:38:40,...,2.0,2.5100,820,61,61,2.033333,2.033333,2.468852,61.0,btw_2and3yrs
2,4,A0270938TP78M39R7CN6,9223372034564167251,2017-12-29,2017-12-29 23:36:51,2019-12-14 00:00:00,5.87,1,F_Over_3yr,2017-09-23 19:38:40,...,3.0,3.6300,820,96,96,3.200000,3.200000,3.403125,35.0,btw_2and3yrs
3,4,A0270938TP78M39R7CN6,9223372034564167251,2018-01-19,2018-01-19 16:45:15,2019-12-14 00:00:00,0.83,1,B_30days_6m,2017-09-23 19:38:40,...,4.0,2.9300,820,117,117,3.900000,3.900000,3.005128,21.0,btw_2and3yrs
4,4,A0270938TP78M39R7CN6,9223372034564167251,2018-01-19,2018-01-19 16:45:49,2019-12-14 00:00:00,0.83,1,B_30days_6m,2017-09-23 19:38:40,...,4.0,3.1375,820,117,117,3.900000,3.900000,3.217949,0.0,btw_2and3yrs


In [8]:
df_concat['promo_dist'].unique()

array(['non_promo', 'vendisto_promo', 'pmd_promo'], dtype=object)

# Analyse different entry points into PMD

In [13]:
def make_final_adjastments(df):
    subset_first_transaction_by_type = df.groupby(['encrypted_customer_id','promo_dist'])['rank_transactions'].min().reset_index()
    subset_first_transaction_by_type.columns = ['encrypted_customer_id','promo_dist_entry','rank_transactions']
    
    copy_dataset = df[["marketplace_id", "encrypted_customer_id","transaction_date_local","revenue","units","content_age",
                   "arpu_monthly_dynamic","frequency","total_units", "total_revenue",
                   "total_units_ttm", "total_revenue_ttm","rank_transactions",
                   "total_units_before_ttm", "total_revenue_before_ttm", "promo_dist",
                   "tenure_years","tenure_up_to_last_purchase_corrected","tenure","is_active_ttm"]]
    df_new = pd.merge(left = copy_dataset, right = subset_first_transaction_by_type, left_on = ['encrypted_customer_id','rank_transactions'],
                 right_on = ['encrypted_customer_id','rank_transactions'], how = 'left')
    
    
    subset_first_transaction_pmd = subset_first_transaction_by_type[subset_first_transaction_by_type['promo_dist_entry'] == 'pmd_promo']
    subset_first_transaction_pmd.columns = ['encrypted_customer_id','promo_dist_entry_specific','rank_transactions_entry']
    
    df_new = pd.merge(left = df_new, right = subset_first_transaction_pmd[['encrypted_customer_id','rank_transactions_entry']],
                      left_on = ['encrypted_customer_id'],
                      right_on = ['encrypted_customer_id'], how = 'left')
    
    df_new['rank_transactions_entry'] = df_new['rank_transactions_entry'].fillna(0)
    
    
    pd.set_option('mode.chained_assignment', None)
    
    df_new['which_relative_to_pmd'] = 'no_entry'
    df_new['which_relative_to_pmd'][(df_new['rank_transactions'] < df_new['rank_transactions_entry']) &
                                   (df_new['rank_transactions_entry'] != 0) ] = "before"
    df_new['which_relative_to_pmd'][(df_new['rank_transactions'] > df_new['rank_transactions_entry']) &
                                   (df_new['rank_transactions_entry'] != 0)] = "after"
    df_new['which_relative_to_pmd'][(df_new['rank_transactions'] == df_new['rank_transactions_entry']) &
                                   (df_new['rank_transactions_entry'] != 0)] = "pmd_entry"
    
    return df_new

In [14]:
df_new = make_final_adjastments(df_concat)

In [15]:
df_new.shape

(9952243, 23)

In [16]:
df_new.columns

Index(['marketplace_id', 'encrypted_customer_id', 'transaction_date_local',
       'revenue', 'units', 'content_age', 'arpu_monthly_dynamic', 'frequency',
       'total_units', 'total_revenue', 'total_units_ttm', 'total_revenue_ttm',
       'rank_transactions', 'total_units_before_ttm',
       'total_revenue_before_ttm', 'promo_dist', 'tenure_years',
       'tenure_up_to_last_purchase_corrected', 'tenure', 'is_active_ttm',
       'promo_dist_entry', 'rank_transactions_entry', 'which_relative_to_pmd'],
      dtype='object')

In [17]:
df_new['which_relative_to_pmd'].unique()

array(['before', 'pmd_entry', 'after'], dtype=object)

In [18]:
def get_diff_stats(x):
    """provides various metrics for count, sum, revenue"""
    d = {}
    d['count_customers'] = x['encrypted_customer_id'].nunique()
    d['arpu_monthly_dynamic'] = x['arpu_monthly_dynamic'].mean()
    d['average_frequency'] = x['frequency'].mean()
    d['units'] = x['units'].sum()
    d['revenue'] = x['revenue'].sum()
    
    return pd.Series(d, index = ['count_customers','arpu_monthly_dynamic', 'average_frequency', 'units','revenue'])

# Subset of those who have done BOTH before and after

In [19]:
def create_subset(df):
    before = df[df['which_relative_to_pmd'].isin(['before'])]['encrypted_customer_id'].unique()
    after_with_before = df[(df['which_relative_to_pmd'] == 'after') &
                           df['encrypted_customer_id'].isin(before)]['encrypted_customer_id'].unique()
    
    subset = df[df['encrypted_customer_id'].isin(after_with_before)]
    
    return subset

In [20]:
d = create_subset(df_new)

In [36]:
d.shape

(8273329, 23)

In [26]:
print(d['encrypted_customer_id'].nunique())
print(d[d['which_relative_to_pmd'] == 'before']['encrypted_customer_id'].nunique())
print(d[d['which_relative_to_pmd'] == 'after']['encrypted_customer_id'].nunique())

547546
547546
547546


In [45]:
def final_output(df):
    grouped = df.groupby(["total_units","tenure_years",
                       "total_units_ttm","rank_transactions_entry","which_relative_to_pmd"]).apply(get_diff_stats).reset_index()
    
    grouped_dist = df.groupby(["total_units","tenure_years",
                       "total_units_ttm","rank_transactions_entry","which_relative_to_pmd","promo_dist"]).apply(get_diff_stats).reset_index()
    
    
    output = pd.merge(left = grouped_dist, right = grouped[['total_units','tenure_years','total_units_ttm',
                                                           'rank_transactions_entry','which_relative_to_pmd','units']],
                     left_on = ['total_units','tenure_years','total_units_ttm',
                                                           'rank_transactions_entry','which_relative_to_pmd'],
                     right_on = ['total_units','tenure_years','total_units_ttm',
                                                           'rank_transactions_entry','which_relative_to_pmd'], how = 'left')
    
    output['share'] = output['units_x'] / output['units_y']
    return grouped, output

In [ ]:
grouped, dist = final_output(d)

In [ ]:
grouped.head()

In [44]:
dist.head()

,total_units,tenure_years,total_units_ttm,rank_transactions_entry,which_relative_to_pmd,promo_dist,count_customers,arpu_monthly_dynamic,average_frequency,units_x,revenue,units_y
0,3,btw_1and2yrs,0.0,2,after,non_promo,454.0,5.127228,53.035242,454.0,2140.21,756.0
1,3,btw_1and2yrs,0.0,2,after,pmd_promo,291.0,3.164539,18.515464,291.0,336.99,756.0
2,3,btw_1and2yrs,0.0,2,after,vendisto_promo,11.0,3.923622,46.545455,11.0,27.55,756.0
3,3,btw_1and2yrs,0.0,2,before,non_promo,746.0,4.694960,0.000000,746.0,3502.44,756.0
4,3,btw_1and2yrs,0.0,2,before,vendisto_promo,10.0,2.750000,0.000000,10.0,27.50,756.0


In [41]:
#grouped.to_csv("grouped_cleanDEReduced.csv")

In [42]:
#dist.to_csv("grouped_by_distDEReduced.csv")